# Import Libraries

In [142]:
import numpy as np
import pandas as pd
pd.set_option("max_columns", None)
from scipy.stats import chi2_contingency
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Import Data

In [143]:
data = pd.read_csv('fashion_data_on_Instagram.csv')
data.set_index('UserId', inplace = True)
data.head()

,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,Comments,CreationTime,Link,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1171579752855683619_212070047,518.0,9840.0,541,abercrombie,High street,"beautiful, summer, fashion, love, cute, food, ...","We were born to be REAL, not to be perfect. ...",https://scontent.cdninstagram.com/t51.2885-15/...,97,0,1453883211,https://www.instagram.com/p/BBCSZPzAMIj/,0.000885,0.336438,0.741220,0.009454,0.001827,0.839466,0.890231,0.292843,0.062638,0.012164,0.931486,3.42211,0.083862,0.015089,0.000583,0.000026,0.000017,0.890586,0.009657,0.000181
1171594777274371222_176762322,7333.0,2300.0,272,abercrombie,High street,"teen, model, brunette, selfie, hollister, snap...","Gotta run, but first, let me take selfie. Me...",https://scontent.cdninstagram.com/t51.2885-15/...,94,0,1453885002,https://www.instagram.com/p/BBCVz4YJEyW/,0.003912,0.995503,0.001728,0.002125,0.002438,0.480598,0.550629,0.063139,0.126848,0.004330,0.466329,2.91971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,91,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,94,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1171508187966229230_2797323089,132.0,456.0,56,abercrombie,High street,"abercrombieandfitch, shopping, love, hollister...",#love #shopping #shoppen #hollister #abercro...,https://scontent.cdninstagram.com/t51.2885-15/...,9,1,1453874680,https://www.instagram.com/p/BBCCH1zMEru/,0.002021,0.014188,0.028351,0.418125,0.601519,0.010840,0.994191,0.332847,0.007911,0.002386,0.393063,1.45586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Clean Data

### Let's See if there are any NaNs in the data

In [17]:
set(data['Hashtags'].isna())

{False, True}

In [18]:
set(data['Caption'].isna())

{False, True}

In [50]:
set(data['Comments '].isna())

{False}

##### So there exist NaN values in Hashtags and Captions. There are no other columns that contain NaN.

In [55]:
data.isna().groupby('Hashtags')['Followings'].count()

Hashtags
False    24606
True       146
Name: Followings, dtype: int64

##### There are 146 NaN rows in hashtags. It is possible that this is simply due to the fact that some posts do not have hashtags.

In [57]:
data.isna().groupby('Caption')['Followings'].count()

Caption
False    24717
True        35
Name: Followings, dtype: int64

##### There are 35 NaN rows in Caption

### Maybe the caption or hashtag information is missing because there is either only a caption or only hashtags....

In [69]:
hashtag_nan = list(data['Hashtags'].isna())
caption_nan = list(data['Caption'].isna())

In [73]:
count = 0
for i in range(len(hashtag_nan)):
    if hashtag_nan[i] and caption_nan[i]:
        count += 1
print('The number of times both hashtag_nan and caption_nan is true is', count)

The number of times both hashtag_nan and caption_nan is true is 18


##### So we have missing hashtag both 18/35 times that we have a missing caption. So it is not exactly correlated. Maybe we can just drop the rows that have missing caption or hashtag if we need to use these, or claim that there was not hashtag or caption for these rows.

### I want to try and see if I can make brand category numeric

In [144]:
brand_category = pd.get_dummies(data['BrandCategory'])

In [145]:
brand_category.head()

,Designer,High street,Mega couture,Small couture
UserId,,,,
1171579752855683619_212070047,0,1,0,0
1171594777274371222_176762322,0,1,0,0
1171407552643586413_581125501,0,1,0,0
1171407552643586413_581125501,0,1,0,0
1171508187966229230_2797323089,0,1,0,0


In [146]:
data_new = pd.concat([data, brand_category], sort = False, axis = 1)
data_new.head()

,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,Comments,CreationTime,Link,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Designer,High street,Mega couture,Small couture
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1171579752855683619_212070047,518.0,9840.0,541,abercrombie,High street,"beautiful, summer, fashion, love, cute, food, ...","We were born to be REAL, not to be perfect. ...",https://scontent.cdninstagram.com/t51.2885-15/...,97,0,1453883211,https://www.instagram.com/p/BBCSZPzAMIj/,0.000885,0.336438,0.741220,0.009454,0.001827,0.839466,0.890231,0.292843,0.062638,0.012164,0.931486,3.42211,0.083862,0.015089,0.000583,0.000026,0.000017,0.890586,0.009657,0.000181,0,1,0,0
1171594777274371222_176762322,7333.0,2300.0,272,abercrombie,High street,"teen, model, brunette, selfie, hollister, snap...","Gotta run, but first, let me take selfie. Me...",https://scontent.cdninstagram.com/t51.2885-15/...,94,0,1453885002,https://www.instagram.com/p/BBCVz4YJEyW/,0.003912,0.995503,0.001728,0.002125,0.002438,0.480598,0.550629,0.063139,0.126848,0.004330,0.466329,2.91971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,91,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,94,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0
1171508187966229230_2797323089,132.0,456.0,56,abercrombie,High street,"abercrombieandfitch, shopping, love, hollister...",#love #shopping #shoppen #hollister #abercro...,https://scontent.cdninstagram.com/t51.2885-15/...,9,1,1453874680,https://www.instagram.com/p/BBCCH1zMEru/,0.002021,0.014188,0.028351,0.418125,0.601519,0.010840,0.994191,0.332847,0.007911,0.002386,0.393063,1.45586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0


##### Now the brand category is represented in the last four columns of the dataframe with one_hot_encoding.

### In order to model with these we need to normalize the variables

In [9]:
for col in data_new.columns:
    if col not in ['BrandName', 'BrandCategory','Hashtags', 'Caption', 'ImgURL', 'Likes', 'Comments ', 'CreationTime','Link','Designer',
       'High street', 'Mega couture', 'Small couture']:
        data_new[col] = data_new[col]/np.linalg.norm(np.array(data_new[col], dtype = float))

# Model with Numeric Data Only

First, lets try to build a model with only the numeric data. This will be easy to do and will allow us to understand whether any of these numeric factors influence the number of likes/comments a user gets. 

### Likes Model

In [71]:
X = data_new[['Followings', 'Followers', 'MediaCount', 'Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor', 'NumberOfPeople',
       'NumberOfFashionProduct', 'Anger', 'Contempt', 'Disgust', 'Fear',
       'Happiness', 'Neutral', 'Sadness', 'Surprise', 'Designer',
       'High street', 'Mega couture', 'Small couture']]
y = data_new['Likes']

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Let's use a Lasso Regression model to regress based on the likes that we receive. I want to use lasso because lasso will send the unimportant variables to zero allowing us to see which features are important. 

In [152]:
lasso = Lasso(alpha = 0.0001, max_iter = 10000).fit(X_train, y_train)

In [153]:
predictions = lasso.predict((X_test))
r2_score(y_test, predictions)

0.2984467039649339

Maybe ridge regression will work better?

In [180]:
ridge = Ridge(alpha = 0.0001).fit(X_train, y_train)

In [181]:
predictions = ridge.predict((X_test))
r2_score(y_test, predictions)

0.29844937687206075

So ridge regression is really no different...

### Try a Deep Learning Model

In [73]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
def our_model():
    model = Sequential()

    model.add(Dense(units=64, activation='relu', input_dim = 27))
    model.add(Dense(units=64, activation='relu', input_dim = 27))
    model.add(Dense(units=64, activation='relu', input_dim = 27))
    model.add(Dense(units=1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [74]:
estimator = KerasRegressor(build_fn=our_model, epochs=100, batch_size=5, verbose=1)

In [75]:
from sklearn.model_selection import KFold, cross_val_score

kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/100
22276/22276 [==============================] - 3s 144us/step - loss: 3987850.1403
Epoch 2/100
22276/22276 [==============================] - 3s 139us/step - loss: 896006.7696
Epoch 3/100
22276/22276 [==============================] - 3s 135us/step - loss: 46557687.4451
Epoch 4/100
22276/22276 [==============================] - 3s 134us/step - loss: 1540440.2986
Epoch 5/100
22276/22276 [==============================] - 3s 134us/step - loss: 1032275.9685
Epoch 6/100
22276/22276 [==============================] - 3s 134us/step - loss: 2056608.9093
Epoch 7/100
22276/22276 [==============================] - 3s 137us/step - loss: 1148519.9571
Epoch 8/100
16325/22276 [====================>.........] - ETA: 0s - loss: 1249694.7402

KeyboardInterrupt: 

Maybe for the neural network model to work we need to normalize the outputs as well??? I have only normalized the input space. Maybe we need to investigate how the input space was normalized. Maybe the categorical variables that are represented by 1's and 0's at the end of the dataframe need to be normalized?

Average loss value for Keras deep learning model based on just numeric values and predicting likes is 154K.

Maybe try using likes / followers?

# Likes to Follower Ratio

First check if there are any null likes or follower entries

In [198]:
data['Likes'].isnull().any() 

False

In [199]:
data['Followers'].isnull().any()

False

In [335]:
data_ver2 = pd.concat([data, brand_category], sort = False, axis = 1)
data_ver2['LikesToFollowers'] = data_ver2['Likes'] / data_ver2['Followers']
data_ver2.head()

,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,Comments,CreationTime,Link,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Designer,High street,Mega couture,Small couture,LikesToFollowers
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1171579752855683619_212070047,518.0,9840.0,541,abercrombie,High street,"beautiful, summer, fashion, love, cute, food, ...","We were born to be REAL, not to be perfect. ...",https://scontent.cdninstagram.com/t51.2885-15/...,97,0,1453883211,https://www.instagram.com/p/BBCSZPzAMIj/,0.000885,0.336438,0.741220,0.009454,0.001827,0.839466,0.890231,0.292843,0.062638,0.012164,0.931486,3.42211,0.083862,0.015089,0.000583,0.000026,0.000017,0.890586,0.009657,0.000181,0,1,0,0,0.009858
1171594777274371222_176762322,7333.0,2300.0,272,abercrombie,High street,"teen, model, brunette, selfie, hollister, snap...","Gotta run, but first, let me take selfie. Me...",https://scontent.cdninstagram.com/t51.2885-15/...,94,0,1453885002,https://www.instagram.com/p/BBCVz4YJEyW/,0.003912,0.995503,0.001728,0.002125,0.002438,0.480598,0.550629,0.063139,0.126848,0.004330,0.466329,2.91971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0.040870
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,91,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0.150413
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,94,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0.155372
1171508187966229230_2797323089,132.0,456.0,56,abercrombie,High street,"abercrombieandfitch, shopping, love, hollister...",#love #shopping #shoppen #hollister #abercro...,https://scontent.cdninstagram.com/t51.2885-15/...,9,1,1453874680,https://www.instagram.com/p/BBCCH1zMEru/,0.002021,0.014188,0.028351,0.418125,0.601519,0.010840,0.994191,0.332847,0.007911,0.002386,0.393063,1.45586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0.019737


In [213]:
data_test = data_ver2[data_ver2['LikesToFollowers'].isnull()]
data_test.head()

,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,Comments,CreationTime,Link,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Designer,High street,Mega couture,Small couture,LikesToFollowers
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1172628647727092971_2860928604,1.0,0.0,5,hollister,High street,"hollister, discountclothes, hollisterco",Hollister pink and white striped sweater. Ne...,https://scontent.cdninstagram.com/t51.2885-15/...,0,0,1454008249,https://www.instagram.com/p/BBGA4rROFTr/,0.000499,0.06023,0.006729,0.698443,0.007520,0.012045,0.533595,0.041080,0.002225,0.000824,0.214680,2.28591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,NaN
1170996282170475597_2603666216,6.0,0.0,3,marcbymarcjacobs,Designer,"marcjacobs, marcbymarcjacobs, watch, love, loveit",#marcjacobs #marcbymarcjacobs #love #loveit ...,https://scontent.cdninstagram.com/t51.2885-15/...,0,0,1453813656,https://www.instagram.com/p/BBANuoYEwRN/,0.008247,0.08745,0.032036,0.759844,0.017295,0.029323,0.936350,0.566902,0.009668,0.020149,0.589774,2.14382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,NaN


Drop rows where LikesToFollowers is NAN. Likes and Followers count at both 0.

In [336]:
data_ver2 = data_ver2.dropna()

In [337]:
data_ver2['LikesToFollowers'].isnull().any()

False

In [338]:
X = data_ver2[['Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor', 'NumberOfPeople',
       'NumberOfFashionProduct', 'Anger', 'Contempt', 'Disgust', 'Fear',
       'Happiness', 'Neutral', 'Sadness', 'Surprise', 'Designer',
       'High street', 'Mega couture', 'Small couture']]
y = data_ver2['LikesToFollowers']

In [339]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [340]:
def standardize(train, test):


    mean = np.mean(train, axis=0)
    std = np.std(train, axis=0)+0.000001

    X_train = (train - mean) / std
    X_test = (test - mean) /std
    return X_train, X_test

In [341]:
X_train, X_test = standardize(X_train, X_test)

In [342]:
X_train

,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Designer,High street,Mega couture,Small couture
UserId,,,,,,,,,,,,,,,,,,,,,,,,
1171435358474800898_2002493650,-0.257829,-0.825983,-0.527356,1.372176,-0.461804,-0.823659,1.134444,2.033331,-0.599170,-0.495454,-0.919080,-1.193306,-0.101418,-0.129131,-0.084296,-0.058561,-0.277616,-0.432297,-0.161836,-0.105340,1.102001,-0.545524,-0.285807,-0.567418
1170371593584137509_1495669756,-0.227933,1.804304,-0.477248,-0.933533,-0.469514,1.625746,-1.086625,-0.791010,1.901818,3.064318,0.799990,1.097663,-0.095371,0.078939,-0.038890,-0.058274,-0.248947,2.635104,-0.136372,-0.099274,1.102001,-0.545524,-0.285807,-0.567418
1171553482977030551_2031918563,-0.258684,1.681969,-0.503979,-0.828043,-0.468880,-0.802391,0.205522,-0.661927,-0.557247,-0.468157,-0.672012,-0.517076,-0.101418,-0.129131,-0.084296,-0.058561,-0.277616,-0.432297,-0.161836,-0.105340,-0.907437,1.833092,-0.285807,-0.567418
1171440949699641276_196263166,-0.258893,-0.818154,-0.487886,1.369692,-0.470000,-0.822627,1.298127,0.349457,-0.598006,-0.501364,-1.036417,-1.024392,-0.101418,-0.129131,-0.084296,-0.058561,-0.277616,-0.432297,-0.161836,-0.105340,1.102001,-0.545524,-0.285807,-0.567418
1171468085370242138_2080653280,-0.254818,-0.749698,-0.520174,-0.840094,3.177836,-0.761215,1.321600,-0.689656,-0.536767,-0.466665,-0.737731,-1.352897,-0.101418,-0.129131,-0.084296,-0.058561,-0.277616,-0.432297,-0.161836,-0.105340,-0.907437,-0.545524,3.498838,-0.567418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171428953965959033_234068,-0.245324,-0.354788,-0.522946,0.743310,-0.387432,-0.809811,-0.940217,-0.529932,-0.563907,-0.457990,-0.715464,-0.902361,-0.101418,-0.129131,-0.084296,-0.058561,-0.277616,-0.432297,-0.161836,-0.105340,-0.907437,1.833092,-0.285807,-0.567418
1168716003269284990_1752602948,-0.228085,1.633713,0.281531,-0.932188,-0.468609,1.623745,-1.207221,-0.734564,1.257544,3.072531,1.199481,1.234805,0.079776,-0.099348,0.048568,-0.055350,-0.215383,2.603526,-0.086380,-0.081278,-0.907437,-0.545524,-0.285807,1.762361
1171093780916631679_13357,2.325838,0.051541,0.178640,-0.929515,-0.356607,1.634504,-0.124527,-0.007779,2.352451,-0.438659,1.766141,0.153108,-0.088203,-0.116059,-0.084097,-0.058229,-0.277424,2.663796,-0.156425,-0.098608,1.102001,-0.545524,-0.285807,-0.567418


In [343]:
y_train.head()

UserId
1171435358474800898_2002493650    0.011494
1170371593584137509_1495669756    0.094961
1171553482977030551_2031918563    0.000271
1171440949699641276_196263166     0.000000
1171468085370242138_2080653280    0.000306
Name: LikesToFollowers, dtype: float64

In [350]:
import keras
keras.backend.floatx()
keras.backend.set_floatx('float32')
keras.backend.floatx()

'float32'

In [352]:
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation, Dropout
from keras.optimizers import Adadelta
from keras.layers.advanced_activations import LeakyReLU

def our_model_ver2():
    model = Sequential()
    model.add(Dense(units=64, input_dim = 24))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.25))

    model.add(Dense(units=64, input_dim = 24))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.5))
    
    model.add(Dense(units=1))
    model.add(LeakyReLU(alpha=0.1))
    opt = Adadelta(clipnorm=0.3)
    model.compile(loss='mean_squared_error', optimizer=opt,metrics=['accuracy'])
    return model

estimator = KerasRegressor(build_fn=our_model_ver2, epochs=5, batch_size=5, verbose=1)

kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/5
14825/14825 [==============================] - 2s 157us/step - loss: nan - accuracy: 0.0017
Epoch 2/5
14825/14825 [==============================] - 2s 145us/step - loss: nan - accuracy: 0.0000e+00
Epoch 3/5
14825/14825 [==============================] - 2s 145us/step - loss: nan - accuracy: 0.0000e+00
Epoch 4/5
12415/14825 [========================>.....] - ETA: 0s - loss: nan - accuracy: 0.0000e+00- ETA: 0s - loss: nan - accuracy: 0.

KeyboardInterrupt: 